In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
%matplotlib inline
import matplotlib.pyplot as plt
from rtree import index

#import models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
# import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ConstantKernel, RBF
#from xgboost import XGBRegressor

from sklearn import model_selection
from sklearn.metrics import explained_variance_score
# from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from tabulate import _table_formats, tabulate
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pygad
import time
import datetime

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from joblib import parallel_backend
import mkl
mkl.set_num_threads(4)

10

In [2]:
datas = './MyData.xlsx'

In [3]:
df=pd.read_excel(datas)
df['logD'] = np.log10(df['diffusivity'])

In [4]:
df2=df[[ 'type', 'gas', 'aperture', 'MetalNum', 'MetalMass', 'size - van der Waals (Å)','mass', 'ascentricF', 'logD', 'size - kinetic diameter (Å)', 'ionicRad', 
       'Μ-N_lff', 'Μ-N_kFF', 'MetalCharge',
       'apertureAtom_σ', 'apertureAtom_e', 'linker_length1', 'linker_length2',
       'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
       'func1_length', 'func2_length', 'func3_length', 'func1_mass',  
       'func2_mass', 'func3_mass', 'func1_charge', 'func2_charge',
       'func3_charge']]

In [5]:
df2=df2.rename(columns={'size - van der Waals (Å)':'diameter', 'size - kinetic diameter (Å)':'kdiameter', 'apertureAtom_e':'e' })

In [6]:
df2 = df2.dropna()
df2=df2.reset_index(drop=True)

In [7]:
x = np.asanyarray(df2[[
    'diameter',
    'mass',
    'ascentricF',
    'kdiameter',
    'ionicRad',
    'MetalNum',
    'MetalMass',
    'apertureAtom_σ',
    'e',
    'linker_length1', 'linker_length2', 'linker_length3',
    'linker_mass1', 'linker_mass2', 'linker_mass3',
    'func1_length', 'func2_length', 'func3_length', 
    'func1_mass', 'func2_mass', 'func3_mass',

                      ]])

y= np.array(df2[['logD']])

In [8]:
df2 = df2.dropna()
df2=df2.reset_index(drop=True)

In [9]:
TypeList = df.type.unique()

In [10]:
TypeList

array(['CdIF-1', 'ZIF-8', 'ZIF-Cu', 'ZIF-67', 'BeIF-1', 'ZIF-7-8',
       'Co-ZIF-7-8', 'Cl-ZIF-7-8', 'Br-ZIF-7-8', 'Be-ZIF-7-8', 'Br-ZIF-8',
       'Im-ZIF-8 (1)', 'Im-ZIF-8 (2)', 'Cd-Cl-ZIF-7-8', 'ZIF8_Mg',
       'ZIF8_Mn', 'Cd-Br-ZIF-7-8', 'Cl-ZIF-8', 'Im-ZIF-8 (3)',
       'Br-ZIF8-Co', 'Co-Br-ZIF-7-8', 'Co-Cl-ZIF-7-8', 'tetr-ZIF8',
       'tetr-ZIF8_Be', 'tetr-ZIF8_Co', 'I-ZIF-7-8', 'F-ZIF-7-8',
       'Co-F-ZIF-7-8', 'Be-F-ZIF-7-8', 'Cu-ZIF-7-8', 'Cu-F-ZIF-7-8',
       'Cu-Cl-ZIF-7-8', 'Cd-I-ZIF-7-8', 'Cd-I-ZIF-7-9', 'Cd-I-ZIF-7-10',
       'Co-I-ZIF-7-8', 'NH2-tetrz', 'NH2_tetrz_Be', 'NH2-tetrz_Co',
       'dClm_Zn', 'dClm_Co', 'dClm_Be', 'dClm_Cd', 'dClm_Mg',
       'Be-I-ZIF-7-8', 'CHO_Zn', 'tetr-ZIF8_Cu', 'dClm_Cu', 'dFm_Zn',
       'dBrm_Zn', 'dIm_Zn', 'Cu-Br-zif-7-8', 'Cu-I-zif-7-8', 'Mg-zif-7-8',
       'Mg-Br-zif-7-8', 'Mg-I-zif-7-8', 'Mn-zif-7-8', 'Mn-Br-zif-7-8',
       'Mn-I-zif-7-8', 'dBrm_Be', 'dBrm_Co', 'dBrm_Cu', 'dBrm_Cd',
       'dBrm_Mg', 'dFm_Co', 'dFm_Cd', 'd

In [40]:
TrainArray = np.delete(TypeList, 0)
TestArray = TypeList[0]

newDf = df2[(df2['type'] == TrainArray[0])]

data_y,data_yhat_XGBR, r2_list = list(), list(), list()

In [41]:
for i in range(1, len(TrainArray)):
    df_b = df2[(df2['type'] == TrainArray[i])]
    newDf = pd.concat([newDf, df_b], axis=0, ignore_index=True)

    testdf = df2[(df2['type'] == TestArray)]

    # print(df_b)
    print(newDf.type.unique())


['ZIF-8' 'ZIF-Cu']
['ZIF-8' 'ZIF-Cu' 'ZIF-67']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8'
 'Br-ZIF-7-8']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8'
 'Br-ZIF-7-8' 'Be-ZIF-7-8']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8'
 'Br-ZIF-7-8' 'Be-ZIF-7-8' 'Br-ZIF-8']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8'
 'Br-ZIF-7-8' 'Be-ZIF-7-8' 'Br-ZIF-8' 'Im-ZIF-8 (1)']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8'
 'Br-ZIF-7-8' 'Be-ZIF-7-8' 'Br-ZIF-8' 'Im-ZIF-8 (1)' 'Im-ZIF-8 (2)']
['ZIF-8' 'ZIF-Cu' 'ZIF-67' 'BeIF-1' 'ZIF-7-8' 'Co-ZIF-7-8' 'Cl-ZIF-7-8'
 'Br-ZIF-7-8' 'Be-ZIF-7-8' 'Br-ZIF-8' 'Im-ZIF-8 (1)' 'Im-ZIF-8 (2)'
 'Cd-Cl-ZIF-7-8']
['ZIF-8' 'ZIF

In [32]:
newDf

,type,gas,aperture,MetalNum,MetalMass,diameter,mass,ascentricF,logD,kdiameter,...,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge
0,CdIF-1,He,3.92,48,112.411,2.66,4.002,-0.390,-7.221849,2.60,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
1,CdIF-1,H2,3.92,48,112.411,2.76,2.010,-0.217,-7.301030,2.89,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
2,CdIF-1,O2,3.92,48,112.411,2.94,31.999,0.022,-8.000000,3.46,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
3,CdIF-1,CO2,3.92,48,112.411,3.24,44.010,0.225,-8.397940,3.30,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
4,CdIF-1,N2,3.92,48,112.411,3.13,28.000,0.037,-8.301030,3.64,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,BeIF-1,butane,2.88,4,9.012,4.52,58.120,0.200,-27.096910,4.50,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1266,-0.1266,-0.1266
1142,BeIF-1,Kr,2.88,4,9.012,4.00,83.800,0.005,-17.204815,3.69,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1266,-0.1266,-0.1266
1143,BeIF-1,Xe,2.88,4,9.012,4.10,131.290,0.008,-25.663540,4.10,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1266,-0.1266,-0.1266
1144,BeIF-1,SF6,2.88,4,9.012,5.13,146.060,0.210,-58.835647,5.10,...,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1266,-0.1266,-0.1266
